In [1]:
import json
from pathlib import Path
import sys

import pandas as pd
from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error
import torch
from torch.utils.data import DataLoader
import wandb

sys.path.insert(0, '../agenticadmet')
from datasets import RegressionDataset
from models import TransformerRegressionModel
from utils import CheckpointParams

[22:55:29] Initializing Normalizer


In [2]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
NUM_WORKERS = 0 # number of workers for dataloader. 0 means using main process for data loading
SMILES_COLUMN = 'smiles_std'
TARGET_COLUMNS = ['LogHLM', 'LogMLM', 'LogD', 'LogKSOL', 'LogMDR1-MDCKII']

MODEL_PARAMS = {
    'config': {
        'vocab_size': 500,
        'hidden_size': 384,
        'num_hidden_layers': 6,
        'num_attention_heads': 8,
        'intermediate_size': 1024,
        'hidden_act': "gelu",
        'hidden_dropout_prob': 0.1,
        'attention_probs_dropout_prob': 0.1,
        'max_position_embeddings': 512,
        'initializer_range': 0.02,
        'layer_norm_eps': 1e-12,
        'pad_token_id': 0,
        'position_embedding_type': "absolute",
        'use_cache': True,
        'type_vocab_size': 2
    },
    'output_dim': len(TARGET_COLUMNS),
    'bias_final': False
}
MODEL_CONFIG = {
    "hidden_dim": 384,
    "num_layers": 1,
    "dropout": 0.12395,
    "weight_decay": 0.0135251,
    "batch_size": 128,
    "max_epochs": 200
}
TOKENIZER_NAME = '<gs_bucket>/artifacts/tokenizers/zinc'
CHECKPOINTS = [
    CheckpointParams(
        path=str(Path('../output/artifacts/mol_mlm_roberta_zinc/last.ckpt').absolute()),
        module_from='roberta',
        module_to='roberta',
        strict=True
    )
]

In [4]:
def prepare_data(input_path, batch_size):
    train_dset = RegressionDataset(
        data_path=input_path,
        smiles_col=SMILES_COLUMN,
        target_cols=TARGET_COLUMNS,
        split='train',
        tokenizer_name=TOKENIZER_NAME,
        mol_masking_prob=0.3,
        mol_masking_val=0.15
    )
    val_dset = RegressionDataset(
        data_path=input_path,
        smiles_col=SMILES_COLUMN,
        target_cols=TARGET_COLUMNS,
        split='val',
        tokenizer_name=TOKENIZER_NAME
    )
    pred_dset = RegressionDataset(
        data_path=input_path,
        smiles_col=SMILES_COLUMN,
        target_cols=TARGET_COLUMNS,
        split=None,
        tokenizer_name=TOKENIZER_NAME,
        remove_all_nan_targets=False
    )

    train_loader = DataLoader(
        train_dset, batch_size=batch_size, shuffle=True,
        num_workers=NUM_WORKERS, collate_fn=train_dset.collate_fn
    )
    val_loader = DataLoader(
        val_dset, batch_size=batch_size, shuffle=False,
        num_workers=NUM_WORKERS, collate_fn=val_dset.collate_fn
    )
    pred_loader = DataLoader(
        pred_dset, batch_size=batch_size, shuffle=False,
        num_workers=NUM_WORKERS, collate_fn=pred_dset.collate_fn
    )

    return train_loader, val_loader, pred_loader

In [5]:
def train_model(config, train_loader, val_loader, save_dir):
    model_params = MODEL_PARAMS.copy()
    model_params['hidden_dim'] = config['hidden_dim']
    model_params['num_layers'] = config['num_layers']
    model_params['dropout'] = config['dropout']
    weight_decay = config['weight_decay']

    model = TransformerRegressionModel(
        model_name='roberta-base',
        model_params=model_params,
        weight_decay=weight_decay,
        checkpoints=CHECKPOINTS
    )

    ckpt_callback = ModelCheckpoint(
        save_top_k=0,
        save_last=True
    )

    exp_name = f"roberta_run_{RUN_IDX}"
    logger = WandbLogger(
        project="admet-challenge",
        name=exp_name,
        prefix=f"{save_dir.stem}",
        save_dir=f"../wandb/{exp_name}"
    )

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=config['max_epochs'], # number of epochs to train for
        # below are needed for Ray and Lightning integration
        enable_progress_bar=True,
        callbacks=[ckpt_callback],
        default_root_dir=save_dir,
        logger=logger
    )

    try:
        trainer.fit(model, train_loader, val_loader)
    except Exception as e:
        logger.finalize("failed")
        wandb.finish()
        raise e
    else:
        logger.finalize("success")

    return model

def predict(model, pred_loader):
    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        enable_progress_bar=True
    )

    model.eval()
    preds = trainer.predict(model, pred_loader, return_predictions=True)
    preds = torch.cat(preds)

    return preds

In [6]:
# copypaste from https://github.com/asapdiscovery/asap-polaris-blind-challenge-examples/blob/a613051bac57060f686d9993e201ecaa15e51009/evaulation.py
# with a log-transform fix according to this issue https://github.com/asapdiscovery/asap-polaris-blind-challenge-examples/issues/14

from collections import defaultdict
from typing import Tuple

def mask_nan(y_true, y_pred):
    mask = ~np.isnan(y_true)
    y_true = np.array(y_true)[mask]
    y_pred = np.array(y_pred)[mask]
    return y_true, y_pred

def eval_admet(preds: dict[str, list], refs: dict[str, list]) -> Tuple[dict[str, float], np.ndarray]:
    """
    Eval ADMET targets with MAE for pre-log10 transformed targets (LogD) and MALE  (MAE on log10 transformed dataset) on non-transformed data

    This provides a "relative" error metric that will not be as sensitive to the large outliers with huge errors. This is sometimes known as MALE.

    Parameters
    ----------
    preds : dict[str, list]
        Dictionary of predicted ADMET values.
    refs : dict[str, list]
        Dictionary of reference ADMET values.

    Returns
    -------
    dict[str, float]
        Returns a dictonary of summary statistics
    """
    keys = {
        "MLM",
        "HLM",
        "KSOL",
        "LogD",
        "MDR1-MDCKII",
    }
    # will be treated as is
    logscale_endpts = {"LogD"}

    collect = defaultdict(dict)

    for k in keys:
        if k not in preds.keys() or k not in refs.keys():
            raise ValueError("required key not present")

        ref, pred = mask_nan(refs[k], preds[k])

        if k in logscale_endpts:
            # already log10scaled
            mae = mean_absolute_error(ref, pred)
            r2 = r2_score(ref, pred)
        else:
            # clip to a detection limit
            # epsilon = 1e-8
            # pred = np.clip(pred, a_min=epsilon, a_max=None)
            # ref = np.clip(ref, a_min=epsilon, a_max=None)

            # transform both log10scale
            pred_log10s = np.log10(pred + 1.)
            ref_log10s = np.log10(ref + 1.)

            # compute MALE and R2 in log space
            mae = mean_absolute_error(ref_log10s, pred_log10s)
            r2 = r2_score(ref_log10s, pred_log10s)

        collect[k]["mean_absolute_error"] = mae
        collect[k]["r2"] = r2

    # compute macro average MAE
    macro_mae = np.mean([collect[k]["mean_absolute_error"] for k in keys])
    collect["aggregated"]["macro_mean_absolute_error"] = macro_mae

    # compute macro average R2
    macro_r2 = np.mean([collect[k]["r2"] for k in keys])
    collect["aggregated"]["macro_r2"] = macro_r2

    return collect

In [7]:
def extract_preds(preds: pd.DataFrame):
    preds_dict = {}
    for t in TARGET_COLUMNS:
        if t in ["LogHLM", "LogMLM", "LogKSOL", "LogMDR1-MDCKII"]:
            # transform back to non-log scale
            preds_dict[t[3:]] = np.power(10, preds.iloc[:, preds.columns.get_loc(f"pred_{t}")].values) - 1.
        else:
            preds_dict[t] = preds.iloc[:, preds.columns.get_loc(f"pred_{t}")].values
    
    return preds_dict

def extract_refs(refs: pd.DataFrame):
    refs_dict = {}
    for t in TARGET_COLUMNS:
        if t in ["LogHLM", "LogMLM", "LogKSOL", "LogMDR1-MDCKII"]:
            refs_dict[t[3:]] = refs.iloc[:, refs.columns.get_loc(t[3:])].values
        else:
            refs_dict[t] = refs.iloc[:, refs.columns.get_loc(t)].values
    
    return refs_dict

In [8]:
def train_and_eval(input_paths, save_dirs):
    for input_path, save_dir in zip(input_paths, save_dirs):
        print(f"Training and predicting on {input_path}")
        train_loader, val_loader, pred_loader = prepare_data(input_path, MODEL_CONFIG['batch_size'])
        model = train_model(MODEL_CONFIG, train_loader, val_loader, save_dir)
        preds = predict(model, pred_loader)

        input_df = pred_loader.dataset.data.copy()
        output_df = input_df.copy()
        output_df[["pred_" + t for t in TARGET_COLUMNS]] = preds
        save_dir.mkdir(parents=True, exist_ok=True)
        output_df.to_csv(save_dir / "predictions.csv", index=False)

        train_preds = extract_preds(output_df[input_df["split"] == "train"])
        train_refs = extract_refs(input_df[input_df["split"] == "train"])
        val_preds = extract_preds(output_df[input_df["split"] == "val"])
        val_refs = extract_refs(input_df[input_df["split"] == "val"])

        metrics = eval_admet(train_preds, train_refs)
        print("Train metrics:")
        print(json.dumps(metrics, indent=2))

        metrics = eval_admet(val_preds, val_refs)
        print("\nVal metrics:")
        print(json.dumps(metrics, indent=2))
    
    wandb.finish()

## Run 0

In [9]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/roberta/run_0/split_{k}') for k in range(5)]
RUN_IDX = 0

In [10]:
train_and_eval(input_paths, save_dirs)

Training and predicting on ../data/asap/datasets/rnd_splits/split_0.csv
0 out of 347 rows are removed due to missing values


0 out of 87 rows are removed due to missing values


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


wandb: Currently logged in as: vladvin111 (vladvin-org) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Train metrics:
{
  "MLM": {
    "mean_absolute_error": 0.10876286814205542,
    "r2": 0.9515041207422992
  },
  "LogD": {
    "mean_absolute_error": 0.14474523508077045,
    "r2": 0.9741172317031266
  },
  "HLM": {
    "mean_absolute_error": 0.11567762011738958,
    "r2": 0.9248294969749418
  },
  "KSOL": {
    "mean_absolute_error": 0.0984356721168935,
    "r2": 0.9538850668233638
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.08288487715447979,
    "r2": 0.9076479171924651
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.11010125452231774,
    "macro_r2": 0.9423967666872393
  }
}

Val metrics:
{
  "MLM": {
    "mean_absolute_error": 0.3428099942534275,
    "r2": 0.11886448094723989
  },
  "LogD": {
    "mean_absolute_error": 0.5579744557359002,
    "r2": 0.6047909147159383
  },
  "HLM": {
    "mean_absolute_error": 0.27409122987229295,
    "r2": 0.5603054158319543
  },
  "KSOL": {
    "mean_absolute_error": 0.32445318441879967,
    "r2": 0.464927790515673
  },
  "MDR1-

split_0-epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇████
split_0-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_epoch,█▆▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,█▄▃▃▂▂▂▂▃▂▂▁
split_0-val/mae,█▇▇▅▃▃▂▂▃▁▁▂▂▂▁▁▁▁▁▂▁▂▁▂▁▁▁▁▁▁▂▁▁▁▂▂▂▁▁▁
split_0-val/r2,▁▂▃▅▇▇▇▇█▇█▇█▇▇▇██▇█▇▇█▇▇▇▇█▇██▇▇▇▇▇▇▇▇▇
split_0-val_loss,█▅▅▄▃▂▂▂▂▂▁▁▁▁▁▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▁
split_1-epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇█
split_1-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_epoch,█▇▆▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▃▃▂▂▃▂▁▂▁▁▁


## Cleaning up + run 1

In [11]:
def clean_data(input_paths, save_dirs, output_dir, remove_worst_pct):
    smiles_to_remove = defaultdict(set)

    for input_path, save_dir in zip(input_paths, save_dirs):
        input_df = pd.read_csv(input_path)
        input_val_df = input_df[input_df["split"] == "val"]
        output_df = pd.read_csv(save_dir / "predictions.csv")
        output_val_df = output_df[input_df["split"] == "val"]

        for t in TARGET_COLUMNS:
            # Sort by absolute error
            notna_mask = input_val_df[t].notna()
            input_val_df = input_val_df[notna_mask]
            output_val_df = output_val_df[notna_mask]

            mae = np.abs(input_val_df[t] - output_val_df[f"pred_{t}"])
            sorted_idx = np.argsort(mae)[::-1]
            smiles_to_remove[t].update(
                input_val_df.iloc[sorted_idx[:int(remove_worst_pct * len(sorted_idx))]]["cxsmiles_std"].tolist()
            )

    for input_path in input_paths:
        input_df = pd.read_csv(input_path)
        for t in TARGET_COLUMNS:
            input_df.loc[input_df["cxsmiles_std"].isin(smiles_to_remove[t]) & (input_df["split"] == "train"), t] = np.nan

        input_df.to_csv(output_dir / input_path.name, index=False)

In [12]:
output_dir = Path("../output/asap/rnd_splits/roberta/run_0/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [13]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [14]:
pd.read_csv(input_paths[0])[TARGET_COLUMNS].isna().sum()

LogHLM            133
LogMLM            131
LogD               90
LogKSOL            69
LogMDR1-MDCKII      9
dtype: int64

In [15]:
pd.read_csv(output_dir / input_paths[0].name)[TARGET_COLUMNS].isna().sum()

LogHLM            181
LogMLM            178
LogD              127
LogKSOL           103
LogMDR1-MDCKII     45
dtype: int64

In [16]:
input_paths = [Path(f'../output/asap/rnd_splits/roberta/run_0/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/roberta/run_1/split_{k}') for k in range(5)]
RUN_IDX = "1_clean_worst_pct_0.2"

In [17]:
train_and_eval(input_paths, save_dirs)

Training and predicting on ../output/asap/rnd_splits/roberta/run_0/cleaned/split_0.csv
0 out of 347 rows are removed due to missing values
0 out of 87 rows are removed due to missing values
Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/roberta_run_1_clean_worst_pct_0.2/wandb/ wasn't writable, using system temp directory.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Train metrics:
{
  "MLM": {
    "mean_absolute_error": 0.20362140202004503,
    "r2": 0.6951793691516915
  },
  "LogD": {
    "mean_absolute_error": 0.11123776211407174,
    "r2": 0.9843844510945366
  },
  "HLM": {
    "mean_absolute_error": 0.20379332378012793,
    "r2": 0.6247000325568115
  },
  "KSOL": {
    "mean_absolute_error": 0.11828276690984624,
    "r2": 0.8589531265959793
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.09103011616648968,
    "r2": 0.860682447018557
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.14559307419811612,
    "macro_r2": 0.8047798852835152
  }
}

Val metrics:
{
  "MLM": {
    "mean_absolute_error": 0.32144245506105096,
    "r2": 0.2876840625678444
  },
  "LogD": {
    "mean_absolute_error": 0.5624028248136694,
    "r2": 0.6068213434395978
  },
  "HLM": {
    "mean_absolute_error": 0.279197786345864,
    "r2": 0.610932316513951
  },
  "KSOL": {
    "mean_absolute_error": 0.28288144963391204,
    "r2": 0.5694549765635838
  },
  "MDR1-MD

split_0-epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇█████
split_0-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_epoch,█▅▄▃▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,█▄▂▂▃▂▂▂▁▂▁▁
split_0-val/mae,█▇▂▂▂▂▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▂▁▁▂▁▁▁▂▂▂
split_0-val/r2,▁▇███▇█▇█▇▇▇▇▇▇▇▇▇▇▇▇████████▇████▇███▇█
split_0-val_loss,█▆▆▅▅▂▂▂▂▁▁▁▁▁▁▂▂▂▁▁▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-epoch,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
split_1-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_epoch,█▇▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▅▃▃▃▂▂▂▂▁▂▁


## Cleaning up + run 2

In [18]:
output_dir = Path("../output/asap/rnd_splits/roberta/run_1/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [19]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [20]:
input_paths = [Path(f'../output/asap/rnd_splits/roberta/run_1/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/roberta/run_2/split_{k}') for k in range(5)]
RUN_IDX = "2_clean_worst_pct_0.2"

In [21]:
train_and_eval(input_paths, save_dirs)

Training and predicting on ../output/asap/rnd_splits/roberta/run_1/cleaned/split_0.csv
0 out of 347 rows are removed due to missing values
0 out of 87 rows are removed due to missing values


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/roberta_run_2_clean_worst_pct_0.2/wandb/ wasn't writable, using system temp directory.


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Train metrics:
{
  "MLM": {
    "mean_absolute_error": 0.23220283115496562,
    "r2": 0.5579025181501113
  },
  "LogD": {
    "mean_absolute_error": 0.14667881902145302,
    "r2": 0.9753080604631285
  },
  "HLM": {
    "mean_absolute_error": 0.20957463870456605,
    "r2": 0.5720501429177555
  },
  "KSOL": {
    "mean_absolute_error": 0.1510905814067655,
    "r2": 0.7902829974296639
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.09787522587678903,
    "r2": 0.7756847749091895
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.16748441923290783,
    "macro_r2": 0.7342456987739696
  }
}

Val metrics:
{
  "MLM": {
    "mean_absolute_error": 0.3149289883912628,
    "r2": 0.3664964434391811
  },
  "LogD": {
    "mean_absolute_error": 0.5223165977001191,
    "r2": 0.5917735931231309
  },
  "HLM": {
    "mean_absolute_error": 0.24672658733242195,
    "r2": 0.6939188304049975
  },
  "KSOL": {
    "mean_absolute_error": 0.2899027299907518,
    "r2": 0.573444011807059
  },
  "MDR1-MD

split_0-epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
split_0-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_epoch,█▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,█▅▅▃▃▃▂▂▁▁▁▁
split_0-val/mae,█▄▃▃▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▁▂▂▂▁▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▂
split_0-val/r2,▁▃▄▄▅▆▆▆▇▇▇▇▇▇▇██▇▇▇▇▇██████▇▇██▇▇██████
split_0-val_loss,█▅▅▄▂▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁
split_1-epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
split_1-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_epoch,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▄▃▃▂▂▂▂▁▂▁▂


## Cleaning up stero impure + run 1

In [22]:
def clean_data(input_paths, save_dirs, output_dir, remove_worst_pct):
    smiles_to_remove = defaultdict(set)

    for input_path, save_dir in zip(input_paths, save_dirs):
        input_df = pd.read_csv(input_path)
        input_val_df = input_df[input_df["split"] == "val"]
        output_df = pd.read_csv(save_dir / "predictions.csv")
        output_val_df = output_df[input_df["split"] == "val"]

        for t in TARGET_COLUMNS:
            # Sort by absolute error
            notna_mask = input_val_df[t].notna()
            input_val_df = input_val_df[notna_mask]
            output_val_df = output_val_df[notna_mask]

            mae = np.abs(input_val_df[t] - output_val_df[f"pred_{t}"])
            sorted_idx = np.argsort(mae)[::-1]
            smiles_to_remove[t].update(
                input_val_df.iloc[sorted_idx[:int(remove_worst_pct * len(sorted_idx))]]["cxsmiles_std"].tolist()
            )

    for input_path in input_paths:
        input_df = pd.read_csv(input_path)
        for t in TARGET_COLUMNS:
            input_df.loc[
                input_df["cxsmiles_std"].isin(smiles_to_remove[t]) & \
                    ~input_df["smiles_ext"].isna() & \
                    (input_df["split"] == "train"),
                t
            ] = np.nan

        input_df.to_csv(output_dir / input_path.name, index=False)

In [23]:
output_dir = Path("../output/asap/rnd_splits/roberta/run_0/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [24]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [25]:
pd.read_csv(input_paths[0])[TARGET_COLUMNS].isna().sum()

LogHLM            200
LogMLM            190
LogD              137
LogKSOL           110
LogMDR1-MDCKII     61
dtype: int64

In [26]:
tmp = pd.read_csv(input_paths[0])
tmp = tmp[tmp["smiles_ext"].isna()]
tmp[TARGET_COLUMNS].isna().sum()

LogHLM            102
LogMLM             80
LogD               70
LogKSOL            57
LogMDR1-MDCKII     36
dtype: int64

In [27]:
pd.read_csv(output_dir / input_paths[0].name)[TARGET_COLUMNS].isna().sum()

LogHLM            208
LogMLM            196
LogD              142
LogKSOL           111
LogMDR1-MDCKII     61
dtype: int64

In [28]:
tmp = pd.read_csv(output_dir / input_paths[0].name)
tmp = tmp[tmp["smiles_ext"].isna()]
tmp[TARGET_COLUMNS].isna().sum()

LogHLM            102
LogMLM             80
LogD               70
LogKSOL            57
LogMDR1-MDCKII     36
dtype: int64

In [29]:
input_paths = [Path(f'../output/asap/rnd_splits/roberta/run_0/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/roberta/run_1/split_{k}') for k in range(5)]
RUN_IDX = "1_clean_worst_pct_0.2_stereo_impure"

In [30]:
train_and_eval(input_paths, save_dirs)

Training and predicting on ../output/asap/rnd_splits/roberta/run_0/cleaned/split_0.csv
0 out of 347 rows are removed due to missing values
0 out of 87 rows are removed due to missing values


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/roberta_run_1_clean_worst_pct_0.2_stereo_impure/wandb/ wasn't writable, using system temp directory.


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Train metrics:
{
  "MLM": {
    "mean_absolute_error": 0.2330355493389978,
    "r2": 0.5824359832012641
  },
  "LogD": {
    "mean_absolute_error": 0.12817867011825243,
    "r2": 0.9799804930789995
  },
  "HLM": {
    "mean_absolute_error": 0.2137776099770363,
    "r2": 0.5799606569032315
  },
  "KSOL": {
    "mean_absolute_error": 0.15514581923002616,
    "r2": 0.7525589099000252
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.10397699091254214,
    "r2": 0.7591974188175785
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.16682292791537096,
    "macro_r2": 0.7308266923802197
  }
}

Val metrics:
{
  "MLM": {
    "mean_absolute_error": 0.2940405465702283,
    "r2": 0.42177831066535565
  },
  "LogD": {
    "mean_absolute_error": 0.49746636841333275,
    "r2": 0.6585083491220253
  },
  "HLM": {
    "mean_absolute_error": 0.24575379373013065,
    "r2": 0.6981647477980601
  },
  "KSOL": {
    "mean_absolute_error": 0.26473626081338797,
    "r2": 0.6071394397847352
  },
  "MDR1

split_0-epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇███
split_0-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_epoch,█▆▅▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,█▅▃▃▂▂▂▁▂▁▁▁
split_0-val/mae,█▆▄▂▂▂▂▁▂▂▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁
split_0-val/r2,▁▅▆▇▇▇▆▇▇▇▇▇██▇▇██▇▇▇█▇▇▇▇▇▇█████▇█████▇
split_0-val_loss,█▇▆▅▃▂▂▂▂▂▁▂▂▂▂▁▁▁▁▁▁▁▁▂▂▁▁▁▂▁▁▁▂▁▂▁▁▁▁▁
split_1-epoch,▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇█████
split_1-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_epoch,█▅▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▄▃▃▂▃▁▁▁▂▁▁


## Removing all stereo impure + run 0

In [31]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/split_{k}.csv') for k in range(5)]
output_dir = Path("../data/asap/datasets/rnd_splits/stereo_pure")
output_dir.mkdir(parents=True, exist_ok=True)

for input_path in input_paths:
    input_df = pd.read_csv(input_path)
    input_df = pd.concat([
        input_df[(input_df["split"] == "train") & input_df["smiles_ext"].isna()],
        input_df[input_df["split"] == "val"]
    ])
    input_df.to_csv(output_dir / input_path.name, index=False)

In [32]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/stereo_pure/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/roberta/run_0/stereo_pure/split_{k}') for k in range(5)]
RUN_IDX = "0_stereo_pure"
train_and_eval(input_paths, save_dirs)

Training and predicting on ../data/asap/datasets/rnd_splits/stereo_pure/split_0.csv
0 out of 166 rows are removed due to missing values
0 out of 87 rows are removed due to missing values


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/roberta_run_0_stereo_pure/wandb/ wasn't writable, using system temp directory.


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

Train metrics:
{
  "MLM": {
    "mean_absolute_error": 0.09458394341728223,
    "r2": 0.9630065104711415
  },
  "LogD": {
    "mean_absolute_error": 0.12565237662588902,
    "r2": 0.9862198108554189
  },
  "HLM": {
    "mean_absolute_error": 0.09821533371843531,
    "r2": 0.9524095980943529
  },
  "KSOL": {
    "mean_absolute_error": 0.12324514450909708,
    "r2": 0.9413972911234435
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.09103032146080825,
    "r2": 0.8689823562532465
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.10654542394630237,
    "macro_r2": 0.9424031133595208
  }
}

Val metrics:
{
  "MLM": {
    "mean_absolute_error": 0.4093814410960141,
    "r2": -0.059534629200242994
  },
  "LogD": {
    "mean_absolute_error": 0.8275039180481073,
    "r2": 0.32661170235489356
  },
  "HLM": {
    "mean_absolute_error": 0.3155491123790569,
    "r2": 0.5560708807569509
  },
  "KSOL": {
    "mean_absolute_error": 0.4905689662301859,
    "r2": 0.13505038953257908
  },
  "M

split_0-epoch,▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇███
split_0-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_epoch,█▇▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁
split_0-train_loss_step,█▃▂▂▁▁▁▁
split_0-val/mae,█▆▁▂▁▁▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▃▂▂▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▂
split_0-val/r2,▁▇███████████▇██▇▇█▇█████▇▇████▇████████
split_0-val_loss,█▂▁▂▂▁▁▁▁▂▂▃▂▂▂▂▁▂▃▃▂▂▃▂▂▂▂▂▃▃▂▂▂▃▃▂▂▁▂▂
split_1-epoch,▁▁▁▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█
split_1-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_epoch,█▇▇▆▆▄▄▃▃▂▂▂▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▆▄▂▄▂▁▂
